In [1]:
import pyslha,itertools
import logging as logger

In [2]:
def equalObjs(obj1,obj2,rtol=0.01,ignore=[]):
    
    if type(obj1) != type(obj2):
        logger.warning("Data types differ (%s,%s)" %(type(obj1),type(obj2)))
        return False
   
    if type(obj1) in [ float, int ]:
        obj1,obj2=float(obj1),float(obj2)
    
    if isinstance(obj1,list):
        obj1 = sorted(obj1)
        obj2 = sorted(obj2)
    
    if obj1 == obj2:
        return True

    if isinstance(obj1,float):
        if obj1 == obj2:
            return True
        diff = 2.*abs(obj1-obj2)/abs(obj1+obj2)
        return diff < rtol
    elif isinstance(obj1,str):
        return obj1.lower() == obj2.lower()
    elif isinstance(obj1,dict):
        for key in obj1:
            if key in ignore:
                continue
            if not key in obj2:
                logger.warning("Key %s missing" %str(key))
                return False
            if not equalObjs(obj1[key],obj2[key],rtol, ignore=ignore ):
                logger.warning( "Dictionaries differ in key %s" %str(key))
                s1,s2 = str(obj1[key]),str(obj2[key]) 
                if len(s1) + len(s2) > 200:
                    logger.warning ( "The values are too long to print." )
                else:
                    logger.warning( 'The values are: %s versus %s'%\
                                ( s1,s2 ) )
                return False
    elif isinstance(obj1,list):
        if len(obj1) != len(obj2):
            logger.warning('Lists differ in length:\n   %i (this run)\n and\n   %i (default)' %\
                                (len(obj1),len(obj2)))
            return False
        for ival,val in enumerate(obj1):
            if not equalObjs(val,obj2[ival],rtol):
                logger.warning('Lists differ:\n   %s (this run)\n and\n   %s (default)' %\
                                (str(val),str(obj2[ival])))
                return False
    else:
        if not equalObjs(obj1.__dict__,obj2.__dict__):
            return False
        
    return True

In [9]:
#Load the SLHA data
slhaFiles = {'andre' : './lhiggs_slha/lhiggs_F1000_13.slha', 'juhi' : './juhi_files/LHT1TeV.slha'}
slhaData = dict([[key,pyslha.readSLHAFile(val)] for key,val in slhaFiles.items()])

In [10]:
#Loop over the files and compare them 2 at a time:
for file1,file2 in itertools.product(slhaData.keys(),slhaData.keys()):
    if file1 == file2:
        continue
    slha1 = slhaData[file1]
    slha2 = slhaData[file2]
    comp = equalObjs(slha1,slha2,rtol=0.01)
    if not comp:
        print('Files %s and %s differ' %(file1,file2))
    else:
        print('Files are equal')

Files are equal
Files are equal


In [11]:
print(sorted(slhaData['andre'].blocks.keys()) ==sorted(slhaData['juhi'].blocks.keys()))

True


In [12]:
print(len(slhaData['juhi'].xsections))
print(len(slhaData['andre'].xsections))

106
106


In [17]:
diffs = []
for pdg in slhaData['juhi'].xsections:
    xsecA = slhaData['juhi'].xsections[pdg].get_xsecs(sqrts=13000.)
    xsecB = slhaData['andre'].xsections[pdg].get_xsecs(sqrts=13000.)
    if len(xsecA) != len(xsecB):
        print(xsecA)
        print(xsecB)
        print('(lens differ)')
    if not xsecA:
        continue
    if xsecA[0].value != xsecB[0].value:
        diffs.append(2.*abs(xsecA[0].value - xsecB[0].value)/abs(xsecA[0].value + xsecB[0].value))
        if diffs[-1] > 0.02:
            print(pdg)
            print(xsecA)
            print(xsecB)
            print('\n')
            

(2212, 2212, 8880022, 8880022)
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 1.75097e-05 pb]
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 1.6273e-05 pb (ufo2slha 1.0)]


(2212, 2212, 8880022, 8880024)
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 0.00043684 pb]
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 0.0004213 pb (ufo2slha 1.0)]


(2212, 2212, -8880024, 8880022)
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 0.000163618 pb]
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 0.00016796 pb (ufo2slha 1.0)]


(2212, 2212, 8880002, 8880002)
[sqrt(s) = 13000 GeV, avg mass scale scheme, Born QCD, Born EW; K_F = 0, K_R = 0, PDF = 0 : xsec = 0.0224316 pb]
[sqrt(s) = 13000 GeV, avg 

In [16]:
print(min(diffs),max(diffs))

(2.337955794844104e-05, 0.08818710575899072)
